# Data augmentation graphs input and output

in this code the augmented data for the chemistry data challange is made.

In [3]:
import ternary
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import comb
import os

# Define image_vertices globally
image_vertices = []

# Function to convert ternary coordinates to image coordinates
def ternary_to_image_coordinates(point, img_size, margins):
    # Unpack the point
    x, y = point

    # Unpack the image size and margins
    img_width, img_height = img_size
    margin_left, margin_right, margin_top, margin_bottom = margins

    # Compute the plot width and height
    plot_width = img_width - margin_left - margin_right
    plot_height = img_height - margin_top - margin_bottom

    # Convert the point to image coordinates
    img_x = margin_left + x * plot_width
    img_y = margin_top + (1 - y) * plot_height  # The y-coordinate is flipped because image coordinates start from the top

    return int(img_x), int(img_y)

# Function to create a Bezier curve
def bezier_curve(points, n=100):
    N = len(points)
    t = np.linspace(0, 1, num=n)

    curve = np.zeros((n, 2))
    for i in range(N):
        curve += np.outer(comb(N - 1, i) * (1 - t) ** (N - 1 - i) * t ** i, points[i])

    return curve

# Function to offset control points
def offset_control_points(points, offset=10):
    offset_points = np.copy(points)

    # Apply the offset to middle control point(s)
    for i in range(1, len(points) - 1):
        x, y = points[i]

        # Determine the closest edge: bottom (x-axis), left (y-axis), or right (y = 100 - x)
        dist_bottom = y
        dist_left = x
        dist_right = x + y - 100

        closest_edge = np.argmin([dist_bottom, dist_left, dist_right])

        if closest_edge == 0:  # Bottom edge
            offset_points[i, 1] -= offset
        elif closest_edge == 1:  # Left edge
            offset_points[i, 0] -= offset
        else:  # Right edge
            offset_points[i] += offset

    return offset_points

# Function to create the ternary plot
def create_ternary_plot(plot_type="input", control_points_list=None):
    global image_vertices  # Access the global variable

    fig, ax = plt.subplots() # creating a new figure with an axes.
    ax.fill([0, 0.5, 1, 0], [0, np.sqrt(3)/2, 0, 0], 'white') # fill triangle in white
    ax.axis('off') # Turn off the axes

    figure, tax = ternary.figure(scale=100, ax=ax) # Pass the axes to the ternary plot
    
    # Define the image size and margins
    img_size = (1000, 1000)  # This should match your actual image size
    margins = (50, 50, 50, 50)  # These should match your actual plot margins

    # Define the ternary diagram vertices
    ternary_vertices = [(0, 0), (1, 0), (0.5, np.sqrt(3)/2)]

    # Convert the vertices to image coordinates
    image_vertices = [ternary_to_image_coordinates(v, img_size, margins) for v in ternary_vertices]

    # If the plot type is "input", generate control points randomly
    if plot_type == "input":
        control_points = np.random.uniform(0, 100, size=(3, 2))
        control_points_list = control_points.tolist()
    else:  # If the plot type is "output", use the provided control points
        control_points = np.array(control_points_list)

    # Offset the control points if the plot type is "output"
    if plot_type == "output":
        control_points = offset_control_points(control_points)

    # Convert the control points to image coordinates
    image_control_points = [ternary_to_image_coordinates(p, img_size, margins) for p in control_points]

    # Create the Bezier curve
    curve = bezier_curve(image_control_points)

    # Plot the control points and the curve
    plt.plot(*zip(*image_control_points), 'ro-')
    plt.plot(*zip(*curve), 'b-')

    # Save the plot
    plt.savefig(f"{plot_type}_ternary_plot.png")
    plt.close()
    
    return control_points_list

# Create a new directory to store the images
if not os.path.exists('images'):
    os.makedirs('images')

# Generate 20 sets of input and output images
for i in range(20):
    # Call the function to create the input plot and get control points
    control_points_list = create_ternary_plot(i, "input")

    # Use the same control points for the output plot
    create_ternary_plot(i, "output", control_points_list)
